#Install requirements in Colab :  
!pip install transformers
!pip install google



In [ ]:
!pip install google
!pip install transformers


Bert-Squad-SEO-Score.ipynb
Anakeyn Bert Squad Score for SEO Alpha 0.1
This tool provide a "Bert Score" for first max 30 pages responding to a question in Google.
This tool is using  Bert-SQuad created by Kamal Raj. We modified  it to calculate a "Bert-Score"
regarding severall documents and not a score inside a unique document.
see original BERT-SQuAD : https://github.com/kamalkraj/BERT-SQuAD

Copyright 2019 Pierre Rouarch 
License GPL V3


First mount drive in colab to access data in Google Drive

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Define directories

In [ ]:
# import os 
# base_dir = 'drive/My Drive/Bert_Squad_SEO'
# print(base_dir)

Modified utils.py (original in BERT-SQuAD by Kamal Raj : https://github.com/kamalkraj/BERT-SQuAD)

In [ ]:

#see original at BERT-SQuAD  by Kamal Raj : #https://github.com/kamalkraj/BERT-SQuAD
#Modifications by Pierre Rouarch  16/12/2019
#beware we change answer content in get_answer see below 


from __future__ import absolute_import, division, print_function

import collections
import math

import numpy as np
import torch
# from transformers.tokenization_bert import (BasicTokenizer,
#                                                     whitespace_tokenize)
from transformers.models.bert.tokenization_bert import BasicTokenizer,whitespace_tokenize
from torch.utils.data import DataLoader, SequentialSampler, TensorDataset


class SquadExample(object):
    """
    A single training/test example for the Squad dataset.
    For examples without an answer, the start and end position are -1.
    """

    def __init__(self,
                 qas_id,
                 question_text,
                 doc_tokens,
                 orig_answer_text=None,
                 start_position=None,
                 end_position=None):
        self.qas_id = qas_id
        self.question_text = question_text
        self.doc_tokens = doc_tokens
        self.orig_answer_text = orig_answer_text
        self.start_position = start_position
        self.end_position = end_position

    def __str__(self):
        return self.__repr__()

    def __repr__(self):
        s = ""
        s += "qas_id: %s" % (self.qas_id)
        s += ", question_text: %s" % (
            self.question_text)
        s += ", doc_tokens: [%s]" % (" ".join(self.doc_tokens))
        if self.start_position:
            s += ", start_position: %d" % (self.start_position)
        if self.end_position:
            s += ", end_position: %d" % (self.end_position)
        return s

class InputFeatures(object):
    """A single set of features of data."""

    def __init__(self,
                 unique_id,
                 example_index,
                 doc_span_index,
                 tokens,
                 token_to_orig_map,
                 token_is_max_context,
                 input_ids,
                 input_mask,
                 segment_ids,
                 paragraph_len,
                 start_position=None,
                 end_position=None,):
        self.unique_id = unique_id
        self.example_index = example_index
        self.doc_span_index = doc_span_index
        self.tokens = tokens
        self.token_to_orig_map = token_to_orig_map
        self.token_is_max_context = token_is_max_context
        self.input_ids = input_ids
        self.input_mask = input_mask
        self.segment_ids = segment_ids
        self.paragraph_len = paragraph_len
        self.start_position = start_position
        self.end_position = end_position

def input_to_squad_example(passage, question):
    """Convert input passage and question into a SquadExample."""

    def is_whitespace(c):
        if c == " " or c == "\t" or c == "\r" or c == "\n" or ord(c) == 0x202F:
            return True
        return False

    paragraph_text = passage
    doc_tokens = []
    char_to_word_offset = []
    prev_is_whitespace = True
    for c in paragraph_text:
        if is_whitespace(c):
            prev_is_whitespace = True
        else:
            if prev_is_whitespace:
                doc_tokens.append(c)
            else:
                doc_tokens[-1] += c
            prev_is_whitespace = False
        char_to_word_offset.append(len(doc_tokens) - 1)

    qas_id = 0
    question_text = question
    start_position = None
    end_position = None
    orig_answer_text = None

    example = SquadExample(
        qas_id=qas_id,
        question_text=question_text,
        doc_tokens=doc_tokens,
        orig_answer_text=orig_answer_text,
        start_position=start_position,
        end_position=end_position)
                
    return example

def _check_is_max_context(doc_spans, cur_span_index, position):
    """Check if this is the 'max context' doc span for the token."""

    # Because of the sliding window approach taken to scoring documents, a single
    # token can appear in multiple documents. E.g.
    #  Doc: the man went to the store and bought a gallon of milk
    #  Span A: the man went to the
    #  Span B: to the store and bought
    #  Span C: and bought a gallon of
    #  ...
    #
    # Now the word 'bought' will have two scores from spans B and C. We only
    # want to consider the score with "maximum context", which we define as
    # the *minimum* of its left and right context (the *sum* of left and
    # right context will always be the same, of course).
    #
    # In the example the maximum context for 'bought' would be span C since
    # it has 1 left context and 3 right context, while span B has 4 left context
    # and 0 right context.
    best_score = None
    best_span_index = None
    for (span_index, doc_span) in enumerate(doc_spans):
        end = doc_span.start + doc_span.length - 1
        if position < doc_span.start:
            continue
        if position > end:
            continue
        num_left_context = position - doc_span.start
        num_right_context = end - position
        score = min(num_left_context, num_right_context) + 0.01 * doc_span.length
        if best_score is None or score > best_score:
            best_score = score
            best_span_index = span_index

    return cur_span_index == best_span_index

def squad_examples_to_features(example, tokenizer, max_seq_length,
                                 doc_stride, max_query_length,cls_token_at_end=False,
                                 cls_token='[CLS]', sep_token='[SEP]', pad_token=0,
                                 sequence_a_segment_id=0, sequence_b_segment_id=1,
                                 cls_token_segment_id=0, pad_token_segment_id=0,
                                 mask_padding_with_zero=True):
    """Loads a data file into a list of `InputBatch`s."""

    unique_id = 1000000000
    # cnt_pos, cnt_neg = 0, 0
    # max_N, max_M = 1024, 1024
    # f = np.zeros((max_N, max_M), dtype=np.float32)
    example_index = 0
    features = []
    # if example_index % 100 == 0:
    #     logger.info('Converting %s/%s pos %s neg %s', example_index, len(examples), cnt_pos, cnt_neg)

    query_tokens = tokenizer.tokenize(example.question_text)

    if len(query_tokens) > max_query_length:
        query_tokens = query_tokens[0:max_query_length]

    tok_to_orig_index = []
    orig_to_tok_index = []
    all_doc_tokens = []
    for (i, token) in enumerate(example.doc_tokens):
        orig_to_tok_index.append(len(all_doc_tokens))
        sub_tokens = tokenizer.tokenize(token)
        for sub_token in sub_tokens:
            tok_to_orig_index.append(i)
            all_doc_tokens.append(sub_token)

    # The -3 accounts for [CLS], [SEP] and [SEP]
    max_tokens_for_doc = max_seq_length - len(query_tokens) - 3

    # We can have documents that are longer than the maximum sequence length.
    # To deal with this we do a sliding window approach, where we take chunks
    # of the up to our max length with a stride of `doc_stride`.
    _DocSpan = collections.namedtuple(  # pylint: disable=invalid-name
        "DocSpan", ["start", "length"])
    doc_spans = []
    start_offset = 0
    while start_offset < len(all_doc_tokens):
        length = len(all_doc_tokens) - start_offset
        if length > max_tokens_for_doc:
            length = max_tokens_for_doc
        doc_spans.append(_DocSpan(start=start_offset, length=length))
        if start_offset + length == len(all_doc_tokens):
            break
        start_offset += min(length, doc_stride)

    for (doc_span_index, doc_span) in enumerate(doc_spans):
        tokens = []
        token_to_orig_map = {}
        token_is_max_context = {}
        segment_ids = []

        # CLS token at the beginning
        if not cls_token_at_end:
            tokens.append(cls_token)
            segment_ids.append(cls_token_segment_id)

        # Query
        for token in query_tokens:
            tokens.append(token)
            segment_ids.append(sequence_a_segment_id)

        # SEP token
        tokens.append(sep_token)
        segment_ids.append(sequence_a_segment_id)

        # Paragraph
        for i in range(doc_span.length):
            split_token_index = doc_span.start + i
            token_to_orig_map[len(tokens)] = tok_to_orig_index[split_token_index]

            is_max_context = _check_is_max_context(doc_spans, doc_span_index,
                                                    split_token_index)
            token_is_max_context[len(tokens)] = is_max_context
            tokens.append(all_doc_tokens[split_token_index])
            segment_ids.append(sequence_b_segment_id)
        paragraph_len = doc_span.length

        # SEP token
        tokens.append(sep_token)
        segment_ids.append(sequence_b_segment_id)

        # CLS token at the end
        if cls_token_at_end:
            tokens.append(cls_token)
            segment_ids.append(cls_token_segment_id)

        input_ids = tokenizer.convert_tokens_to_ids(tokens)

        # The mask has 1 for real tokens and 0 for padding tokens. Only real
        # tokens are attended to.
        input_mask = [1 if mask_padding_with_zero else 0] * len(input_ids)

        # Zero-pad up to the sequence length.
        while len(input_ids) < max_seq_length:
            input_ids.append(pad_token)
            input_mask.append(0 if mask_padding_with_zero else 1)
            segment_ids.append(pad_token_segment_id)

        assert len(input_ids) == max_seq_length
        assert len(input_mask) == max_seq_length
        assert len(segment_ids) == max_seq_length

        start_position = None
        end_position = None

        features.append(
            InputFeatures(
                unique_id=unique_id,
                example_index=example_index,
                doc_span_index=doc_span_index,
                tokens=tokens,
                token_to_orig_map=token_to_orig_map,
                token_is_max_context=token_is_max_context,
                input_ids=input_ids,
                input_mask=input_mask,
                segment_ids=segment_ids,
                paragraph_len=paragraph_len,
                start_position=start_position,
                end_position=end_position))
        unique_id += 1

    return features

def to_list(tensor):
    return tensor.detach().cpu().tolist()

def _get_best_indexes(logits, n_best_size):
    """Get the n-best logits from a list."""
    index_and_score = sorted(enumerate(logits), key=lambda x: x[1], reverse=True)

    best_indexes = []
    for i in range(len(index_and_score)):
        if i >= n_best_size:
            break
        best_indexes.append(index_and_score[i][0])
    return best_indexes

RawResult = collections.namedtuple("RawResult",["unique_id", "start_logits", "end_logits"])

def get_final_text(pred_text, orig_text, do_lower_case, verbose_logging=False):
    """Project the tokenized prediction back to the original text."""

    # When we created the data, we kept track of the alignment between original
    # (whitespace tokenized) tokens and our WordPiece tokenized tokens. So
    # now `orig_text` contains the span of our original text corresponding to the
    # span that we predicted.
    #
    # However, `orig_text` may contain extra characters that we don't want in
    # our prediction.
    #
    # For example, let's say:
    #   pred_text = steve smith
    #   orig_text = Steve Smith's
    #
    # We don't want to return `orig_text` because it contains the extra "'s".
    #
    # We don't want to return `pred_text` because it's already been normalized
    # (the SQuAD eval script also does punctuation stripping/lower casing but
    # our tokenizer does additional normalization like stripping accent
    # characters).
    #
    # What we really want to return is "Steve Smith".
    #
    # Therefore, we have to apply a semi-complicated alignment heuristic between
    # `pred_text` and `orig_text` to get a character-to-character alignment. This
    # can fail in certain cases in which case we just return `orig_text`.

    def _strip_spaces(text):
        ns_chars = []
        ns_to_s_map = collections.OrderedDict()
        for (i, c) in enumerate(text):
            if c == " ":
                continue
            ns_to_s_map[len(ns_chars)] = i
            ns_chars.append(c)
        ns_text = "".join(ns_chars)
        return (ns_text, ns_to_s_map)

    # We first tokenize `orig_text`, strip whitespace from the result
    # and `pred_text`, and check if they are the same length. If they are
    # NOT the same length, the heuristic has failed. If they are the same
    # length, we assume the characters are one-to-one aligned.
    tokenizer = BasicTokenizer(do_lower_case=do_lower_case)

    tok_text = " ".join(tokenizer.tokenize(orig_text))

    start_position = tok_text.find(pred_text)
    if start_position == -1:
        return orig_text
    end_position = start_position + len(pred_text) - 1

    (orig_ns_text, orig_ns_to_s_map) = _strip_spaces(orig_text)
    (tok_ns_text, tok_ns_to_s_map) = _strip_spaces(tok_text)

    if len(orig_ns_text) != len(tok_ns_text):
        return orig_text

    # We then project the characters in `pred_text` back to `orig_text` using
    # the character-to-character alignment.
    tok_s_to_ns_map = {}
    for (i, tok_index) in tok_ns_to_s_map.items():
        tok_s_to_ns_map[tok_index] = i

    orig_start_position = None
    if start_position in tok_s_to_ns_map:
        ns_start_position = tok_s_to_ns_map[start_position]
        if ns_start_position in orig_ns_to_s_map:
            orig_start_position = orig_ns_to_s_map[ns_start_position]

    if orig_start_position is None:
        return orig_text

    orig_end_position = None
    if end_position in tok_s_to_ns_map:
        ns_end_position = tok_s_to_ns_map[end_position]
        if ns_end_position in orig_ns_to_s_map:
            orig_end_position = orig_ns_to_s_map[ns_end_position]

    if orig_end_position is None:
        return orig_text

    output_text = orig_text[orig_start_position:(orig_end_position + 1)]
    return output_text

def _compute_softmax(scores):
    """Compute softmax probability over raw logits."""
    if not scores:
        return []

    max_score = None
    for score in scores:
        if max_score is None or score > max_score:
            max_score = score

    exp_scores = []
    total_sum = 0.0
    for score in scores:
        x = math.exp(score - max_score)
        exp_scores.append(x)
        total_sum += x
    probs = []
    for score in exp_scores :
        probs.append(score / total_sum)
    return probs

def get_answer(example, features, all_results, n_best_size,
                max_answer_length, do_lower_case):
    example_index_to_features = collections.defaultdict(list)
    for feature in features:
        example_index_to_features[feature.example_index].append(feature)
    
    unique_id_to_result = {}
    for result in all_results:
        unique_id_to_result[result.unique_id] = result
    
    _PrelimPrediction = collections.namedtuple( "PrelimPrediction",["feature_index", "start_index", "end_index", "start_logit", "end_logit"])

    example_index = 0
    features = example_index_to_features[example_index]

    prelim_predictions = []

    for (feature_index, feature) in enumerate(features):
        result = unique_id_to_result[feature.unique_id]
        start_indexes = _get_best_indexes(result.start_logits, n_best_size)
        end_indexes = _get_best_indexes(result.end_logits, n_best_size)
        for start_index in start_indexes:
            for end_index in end_indexes:
                # We could hypothetically create invalid predictions, e.g., predict
                # that the start of the span is in the question. We throw out all
                # invalid predictions.
                if start_index >= len(feature.tokens):
                    continue
                if end_index >= len(feature.tokens):
                    continue
                if start_index not in feature.token_to_orig_map:
                    continue
                if end_index not in feature.token_to_orig_map:
                    continue
                if not feature.token_is_max_context.get(start_index, False):
                    continue
                if end_index < start_index:
                    continue
                length = end_index - start_index + 1
                if length > max_answer_length:
                    continue
                prelim_predictions.append(
                    _PrelimPrediction(
                        feature_index=feature_index,
                        start_index=start_index,
                        end_index=end_index,
                        start_logit=result.start_logits[start_index],
                        end_logit=result.end_logits[end_index]))
    prelim_predictions = sorted(prelim_predictions,key=lambda x: (x.start_logit + x.end_logit),reverse=True)
    _NbestPrediction = collections.namedtuple("NbestPrediction",
                        ["text", "start_logit", "end_logit","start_index","end_index"])
    seen_predictions = {}
    nbest = []
    for pred in prelim_predictions:
        if len(nbest) >= n_best_size:
            break
        feature = features[pred.feature_index]
        orig_doc_start = -1
        orig_doc_end = -1
        if pred.start_index > 0:  # this is a non-null prediction
            tok_tokens = feature.tokens[pred.start_index:(pred.end_index + 1)]
            orig_doc_start = feature.token_to_orig_map[pred.start_index]
            orig_doc_end = feature.token_to_orig_map[pred.end_index]
            orig_tokens = example.doc_tokens[orig_doc_start:(orig_doc_end + 1)]
            tok_text = " ".join(tok_tokens)

            # De-tokenize WordPieces that have been split off.
            tok_text = tok_text.replace(" ##", "")
            tok_text = tok_text.replace("##", "")

            # Clean whitespace
            tok_text = tok_text.strip()
            tok_text = " ".join(tok_text.split())
            orig_text = " ".join(orig_tokens)

            final_text = get_final_text(tok_text, orig_text,do_lower_case)
            if final_text in seen_predictions:
                continue

            seen_predictions[final_text] = True
        else:
            final_text = ""
            seen_predictions[final_text] = True

        nbest.append(
            _NbestPrediction(
                text=final_text,
                start_logit=pred.start_logit,
                end_logit=pred.end_logit,
                start_index=orig_doc_start,
                end_index=orig_doc_end))
        
    #End for for pred in prelim_predictions

    if not nbest:
        nbest.append(_NbestPrediction(text="empty", start_logit=0.0, end_logit=0.0,start_index=-1,
                end_index=-1))

    assert len(nbest) >= 1


   #PR get values in separate lists
    total_scores = []
    answers = []
    starts = []
    ends = []
    for entry in nbest:
        total_scores.append(entry.start_logit + entry.end_logit)
        answers.append(entry.text)
        starts.append(entry.start_index)
        ends.append(entry.end_index)
        
    #relative to the document  sum(local_probs)==1  we keep it
    probs = _compute_softmax(total_scores)    
        
    #PR Total_probs create from total_scores (which is a logit)
    total_probs = [] 
    for score in total_scores :  #total score = 
        total_probs.append(1/(1+math.exp(-score)))  #transform total_scores in total_probs
    mean_total_prob = np.mean(total_probs )    #the new bert score indicator !!!

    
   
    #PR change answer outputs
    
    answer = {"answers" : answers,  #responses   texts 
               "starts" : starts,   #Start indexes of responses
               "ends" : ends,    #Start indexes  responses
               "doc_tokens" : example.doc_tokens,  #document tokens
               "local_probs" : probs,  #all best local probs (old indicators or results after softmax)
               "total_scores" :total_scores,   #All best scores (not softmaxed)
               "total_probs" : total_probs,    #All best probs  (not softmaxed)
               "mean_total_prob" : mean_total_prob  #the new bert score indicator !!!
             }
    return answer

Modified Bert.py (original in  BERT-SQuAD  by Kamal Raj : #https://github.com/kamalkraj/BERT-SQuAD)

In [ ]:

#see original at BERT-SQuAD  by Kamal Raj : #https://github.com/kamalkraj/BERT-SQuAD
#Modifications by Pierre Rouarch  16/12/2019
#We just add a parameter n_best_size in class QA definition.
#beware we change answer content in get_answer (see in utils.py)


from __future__ import absolute_import, division, print_function

import collections
import logging
import math

import numpy as np
import torch
from transformers import (WEIGHTS_NAME, BertConfig,
                                  BertForQuestionAnswering, BertTokenizer)
from transformers import WEIGHTS_NAME
#from transformers.models.bert.modeling_bert import BertTokenizer
from torch.utils.data import DataLoader, SequentialSampler, TensorDataset

#from utils import (get_answer, input_to_squad_example, squad_examples_to_features, to_list)

RawResult = collections.namedtuple("RawResult",
                                   ["unique_id", "start_logits", "end_logits"])



class QA:

    def __init__(self,model_path: str, n_best_size=20):
        self.max_seq_length = 384 #original 384
        self.doc_stride = 1000#128 
        self.do_lower_case = True
        self.max_query_length =500# orig 64
        self.n_best_size =  n_best_size #orig 20  #Change by PR
        self.max_answer_length = 300 #defino o tamanho da resposta
        self.model, self.tokenizer = self.load_model(model_path)
        if torch.cuda.is_available():
            self.device = 'cuda'
        else:
            self.device = 'cpu'
        self.model.to(self.device)
        self.model.eval()


    def load_model(self,model_path: str,do_lower_case=False):
        #config = BertConfig.from_pretrained(model_path + "/bert_config.json")
        #tokenizer = BertTokenizer.from_pretrained(model_path, do_lower_case=do_lower_case)
        #model = BertForQuestionAnswering.from_pretrained(model_path, from_tf=False, config=config)
        #PR use a standard  https://huggingface.co/transformers/pretrained_models.html  SQuAD fine Tune Model
        tokenizer = BertTokenizer.from_pretrained(model_path)
        model = BertForQuestionAnswering.from_pretrained(model_path)
        return model, tokenizer
    
    def predict(self,passage :str,question :str):
        example = input_to_squad_example(passage,question)
        features = squad_examples_to_features(example,self.tokenizer,self.max_seq_length,self.doc_stride,self.max_query_length)
        all_input_ids = torch.tensor([f.input_ids for f in features], dtype=torch.long)
        all_input_mask = torch.tensor([f.input_mask for f in features], dtype=torch.long)
        all_segment_ids = torch.tensor([f.segment_ids for f in features], dtype=torch.long)
        all_example_index = torch.arange(all_input_ids.size(0), dtype=torch.long)
        dataset = TensorDataset(all_input_ids, all_input_mask, all_segment_ids,
                                all_example_index)
        eval_sampler = SequentialSampler(dataset)
        eval_dataloader = DataLoader(dataset, sampler=eval_sampler, batch_size=1)
        all_results = []
        for batch in eval_dataloader:
            batch = tuple(t.to(self.device) for t in batch)
            with torch.no_grad():
                inputs = {'input_ids':      batch[0],
                        'attention_mask': batch[1],
                        'token_type_ids': batch[2]  
                        }
                example_indices = batch[3]
                outputs = self.model(**inputs)

            for i, example_index in enumerate(example_indices):
                eval_feature = features[example_index.item()]
                unique_id = int(eval_feature.unique_id)
                result = RawResult(unique_id    = unique_id,
                                    start_logits = to_list(outputs[0][i]),
                                    end_logits   = to_list(outputs[1][i]))
                all_results.append(result)

        answer = get_answer(example,features,all_results,self.n_best_size,self.max_answer_length,self.do_lower_case)
        return answer


Now choose an expression / question / keyword to ask to Google

In [ ]:
myKeyword="o que é computação hiperconvergente"

Next load Bert-Squad Model (modified version)

In [ ]:
#from bert import QA
n_best_size = 20 #estava 50

#list of pretrained model
#https://huggingface.co/transformers/pretrained_models.html
#!!!!!  instantiate a model  fine tuned on SQuAD
#Choose your model
#'bert-large-uncased-whole-word-masking-finetuned-squad'
#'bert-large-cased-whole-word-masking-finetuned-squad'
model = QA('pierreguillou/bert-large-cased-squad-v1.1-portuguese', n_best_size) #bert-base-multilingual-cased para portugues, bert-large-uncased-whole-word-masking-finetuned-squad



In [ ]:
import pickle

In [ ]:
filename = 'finalized_model.sav'
pickle.dump(model, open(filename, 'wb'))
 
# some time later...
 
# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))
# result = loaded_model.score(X_test, Y_test)
# print(result)

Import needed libraries and functions

In [ ]:
 #import needed libraries
import pandas as pd
import numpy as np
#pip install google  #to install Google Search by Mario Vilas see
#https://python-googlesearch.readthedocs.io/en/latest/
import googlesearch  #Scrap serps
#to randomize pause
import random
import time  #to calcute page time downlod
from datetime import date      
import sys #for sys variables

import requests #to read urls contents
from bs4 import BeautifulSoup  #to decode html
from bs4.element import Comment

#remove comments and non visible tags
def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True


###############################################
#Search in Google  and scrap Urls 
###############################################

In [ ]:
dfScrap = pd.DataFrame(columns=['keyword', 'page', 'position', 'BERT_score', 'source', 'search_date'])
myNum=10
myStart=0
myStop=10 #get by ten
myMaxStart=10  #only 30 pages
myLowPause=5
myHighPause=15
myDate=date.today()
nbTrials = 0
myTLD = "com"   #Google tld   -> we search in google.com
myHl = "pt" #in english
#this may be long 
while myStart <  myMaxStart:
    print("PASSAGE NUMBER :"+str(myStart))
    print("Query:"+myKeyword)
    #change user-agent and pause to avoid blocking by Google
    myPause = random.randint(myLowPause,myHighPause)  #long pause
    print("Pause:"+str(myPause))
    #change user_agent  and provide local language in the User Agent
    #myUserAgent =  getRandomUserAgent(myconfig.userAgentsList, myUserAgentLanguage) 
    myUserAgent = googlesearch.get_random_user_agent()
    print("UserAgent:"+str(myUserAgent))
    #myPause=myPause*(nbTrials+1)  #up the pause if trial get nothing
    #print("Pause:"+str(myPause))
    try  : 
         urls = googlesearch.search(query=myKeyword, tld=myTLD, lang=myHl, safe='off', 
                                                   num=myNum, start=myStart, stop=myStop, domains=None, pause=myPause, 
                                                   tpe='', user_agent=myUserAgent)
         
         df = pd.DataFrame(columns=['keyword', 'page', 'position', 'BERT_score', 'source', 'search_date'])
         for url in urls :
             print("URL:"+url)
             df.loc[df.shape[0],'page'] = url
         df['keyword'] = myKeyword  #fill with current keyword
        # df['tldLang'] = myTLDLang  #fill with current country / tld lang not use here all in in english
         df['position'] = df.index.values + 1 + myStart #position = index +1 + myStart
         df['BERT_score'] = 0 #not yet calculate
         df['source'] = "Scrap"  #fill with source origin  here scraping Google 
         #other potentials options : Semrush, Yooda Insight...
         df['search_date'] = myDate
         dfScrap = pd.concat([dfScrap, df], ignore_index=True) #concat scraps
         # time.sleep(myPause) #add another  pause
         if (df.shape[0] > 0):
             nbTrials = 0
             myStart += 10
         else :
             nbTrials +=1
             if (nbTrials > 3) :
                 nbTrials = 0
                 myStart += 10
             #myStop += 10
    except :
        exc_type, exc_value, exc_traceback = sys.exc_info()
        print("ERROR")
        print(exc_type.__name__)
        print(exc_value)
        print(exc_traceback)
       # time.sleep(600) #add a big pause if you get an error.
       #/while myStart <  myMaxStart:


dfScrap.info()
dfScrapUnique=dfScrap.drop_duplicates()  #on enlève les evéntuels doublons
dfScrapUnique.info()
#Sauvegarde 
# dfScrapUnique.to_csv("dfScrapUniqueJN.csv", sep=";", encoding='utf-8', index=False)  #séparateur ; 
# dfScrapUnique.to_json("dfScrapUniqueJN.json")

PASSAGE NUMBER :0
Query:o que é computação hiperconvergente
Pause:7
UserAgent:b'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:29.0) Gecko/20120101 Firefox/29.0'
PASSAGE NUMBER :0
Query:o que é computação hiperconvergente
Pause:5
UserAgent:b'Mozilla/5.0 (X11; U; Linux i686; en-US; rv:1.8.1.11) Gecko/20071204 Ubuntu/7.10 (gutsy) Firefox/2.0.0.11'
URL:https://www.redhat.com/pt-br/topics/hyperconverged-infrastructure/what-is-hyperconverged-infrastructure
URL:https://www.cisco.com/c/m/pt_br/products/data-center/what-is-hyperconverged-infrastructure.html
URL:https://www.cisco.com/c/pt_pt/solutions/data-center-virtualization/what-is-hyperconverged-infrastructure.html
URL:https://www.bravotecnologia.com.br/hiperconvergencia-o-que-e/
URL:https://www.vmware.com/br/products/hyper-converged-infrastructure.html
URL:https://www.nutanix.com/br/hyperconverged-infrastructure
URL:https://netconsulting.com.br/o-que-e-infraestrutura-hiperconvergente/
URL:https://www.binarionet.com.br/infraestrutura-hiperconverge

SCRAP URLS

In [ ]:
###### filter extensions
extensionsToCheck = ('.7z','.aac','.au','.avi','.bmp','.bzip','.css','.doc',
                                     '.docx','.flv','.gif','.gz','.gzip','.ico','.jpg','.jpeg',
                                     '.js','.mov','.mp3','.mp4','.mpeg','.mpg','.odb','.odf',
                                     '.odg','.odp','.ods','.odt','.pdf','.png','.ppt','.pptx',
                                     '.psd','.rar','.swf','.tar','.tgz','.txt','.wav','.wmv',
                                     '.xls','.xlsx','.xml','.z','.zip')

indexGoodFile= dfScrapUnique ['page'].apply(lambda x : not x.endswith(extensionsToCheck) )
dfUrls2=dfScrapUnique.iloc[indexGoodFile.values]
dfUrls2.reset_index(inplace=True, drop=True)
dfUrls2.info()
                
#######################################################
# Scrap Urls only one time
########################################################
myPagesToScrap = dfUrls2['page'].unique()
dfPagesToScrap= pd.DataFrame(myPagesToScrap, columns=["page"])
#dfPagesToScrap.size #9
#add new variables
dfPagesToScrap['statusCode'] = np.nan
dfPagesToScrap['html'] = ''  #
dfPagesToScrap['encoding'] = ''  #
dfPagesToScrap['elapsedTime'] = np.nan

for i in range(0,len(dfPagesToScrap)) :
    url = dfPagesToScrap.loc[i, 'page']
    print("Page i = "+url+" "+str(i))
    startTime = time.time()
    try:
        #html = urllib.request.urlopen(url).read()$
         r = requests.get(url,timeout=(5, 14))  #request
         dfPagesToScrap.loc[i,'statusCode'] = r.status_code
         print('Status_code '+str(dfPagesToScrap.loc[i,'statusCode']))
         if r.status_code == 200. :   #can't decode utf-7
             print("Encoding="+str(r.encoding))
             dfPagesToScrap.loc[i,'encoding'] = r.encoding
             if   r.encoding == 'UTF-7' :  #don't get utf-7 content pb with db
                 dfPagesToScrap.loc[i, 'html'] =""
                 print("UTF-7 ok page ") 
             else :
                dfPagesToScrap.loc[i, 'html'] = r.text
                #au format texte r.text - pas bytes : r.content
                print("ok page ") 
                #print(dfPagesToScrap.loc[i, 'html'] )
    except:
        print("Error page requests ") 
                
    endTime= time.time()   
    dfPagesToScrap.loc[i, 'elapsedTime'] =  endTime - startTime
    #/
    
dfPagesToScrap.info()
        
#merge dfUrls2, dfPagesToScrap  -> dfUrls3
dfUrls3 = pd.merge(dfUrls2, dfPagesToScrap, on='page', how='left') 
#keep only  status code = 200   
dfUrls3 = dfUrls3.loc[dfUrls3['statusCode'] == 200]  
#dfUrls3 = dfUrls3.loc[dfUrls3['encoding'] != 'UTF-7']   #can't save utf-7  content in db ????
dfUrls3 = dfUrls3.loc[dfUrls3['html'] != ""] #don't get empty html
dfUrls3.reset_index(inplace=True, drop=True)    
dfUrls3.info() # 
dfUrls3 = dfUrls3.dropna()  #remove rows with at least one na
dfUrls3.reset_index(inplace=True, drop=True) 
dfUrls3.info() #

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   keyword      10 non-null     object
 1   page         10 non-null     object
 2   position     10 non-null     object
 3   BERT_score   10 non-null     object
 4   source       10 non-null     object
 5   search_date  10 non-null     object
dtypes: object(6)
memory usage: 608.0+ bytes
Page i = https://www.redhat.com/pt-br/topics/hyperconverged-infrastructure/what-is-hyperconverged-infrastructure 0
Status_code 200.0
Encoding=utf-8
ok page 
Page i = https://www.cisco.com/c/m/pt_br/products/data-center/what-is-hyperconverged-infrastructure.html 1
Status_code 200.0
Encoding=utf-8
ok page 
Page i = https://www.cisco.com/c/pt_pt/solutions/data-center-virtualization/what-is-hyperconverged-infrastructure.html 2
Status_code 200.0
Encoding=utf-8
ok page 
Page i = https://www.bravotecnologia.com.br/hiperconverge

Calculate BERT Score 

In [ ]:
#Remove Duplicates before calculate Bert Score
dfPagesUnique = dfUrls3.drop_duplicates(subset='page')  #remove duplicate's pages
dfPagesUnique = dfPagesUnique.dropna()  #remove na
dfPagesUnique.reset_index(inplace=True, drop=True) #reset index


#Create body from HTML and get Bert_score ### may be long 

dfPagesAnswers = pd.DataFrame(columns=['keyword', 'page', 'position', 'BERT_score', 'source', 'search_date','answers','starts', 'ends', 'local_probs', 'total_probs'])


for i in range(0,len(dfPagesUnique)) :
    soup="" 
    print("Page keyword tldLang  i = "+ dfPagesUnique.loc[i, 'page']+" "+ dfPagesUnique.loc[i, 'keyword']+" "+str(i))
    encoding = dfPagesUnique.loc[i, 'encoding'] #get previously
    print("get body content encoding"+encoding)
    try:
        soup = BeautifulSoup( dfPagesUnique.loc[i, 'html'], 'html.parser')
    except :
        soup="" 
               
    if len(soup) != 0 :
        #TBody Content
        texts = soup.findAll(text=True)
        visible_texts = filter(tag_visible, texts)  
        myBody = " ".join(t.strip() for t in visible_texts)
        myBody=myBody.strip()
        #myBody = strip_accents(myBody, encoding).lower()  #think  to do a global clean instead
        myBody=" ".join(myBody.split(" "))  #remove multiple spaces
        print(myBody)
        dfPagesUnique.loc[i, 'body'] = myBody
        #answer = model.predict( dfPagesUnique.loc[i, 'body'],myKeyword)
        answer = loaded_model.predict( dfPagesUnique.loc[i, 'body'],myKeyword)
        display(answer)
        #print("BERT_score"+str(answer['mean_total_prob']))
        dfPagesUnique.loc[i,  'BERT_score'] = answer['mean_total_prob']
        dfAnswer = pd.DataFrame(answer,  columns=['answers','starts', 'ends', 'local_probs', 'total_probs'])
        dfPageAnswer = pd.DataFrame(columns=['keyword', 'page', 'position', 'BERT_score', 'source', 'search_date','answers','starts', 'ends', 'local_probs', 'total_probs'])
        for k in range (0, len(dfAnswer)) :
            dfPageAnswer.loc[k, 'keyword'] = dfPagesUnique.loc[i,  'keyword']
            dfPageAnswer.loc[k, 'page'] = dfPagesUnique.loc[i,  'page']
            dfPageAnswer.loc[k, 'position'] = dfPagesUnique.loc[i,  'position']
            dfPageAnswer.loc[k,'BERT_score'] = dfPagesUnique.loc[i, 'BERT_score']
            dfPageAnswer.loc[k,'source'] = dfPagesUnique.loc[i,'source']
            dfPageAnswer.loc[k,'search_date'] = dfPagesUnique.loc[i,'search_date']
            dfPageAnswer.loc[k,'answers'] = dfAnswer.loc[k,'answers']
            dfPageAnswer.loc[k,'starts'] = dfAnswer.loc[k,'starts']
            dfPageAnswer.loc[k,'ends'] = dfAnswer.loc[k,'ends']
            dfPageAnswer.loc[k, 'local_probs'] = dfAnswer.loc[k, 'local_probs']
            dfPageAnswer.loc[k, 'total_probs'] = dfAnswer.loc[k, 'total_probs']
        dfPagesAnswers = pd.concat([dfPagesAnswers, dfPageAnswer], ignore_index=True) #concat Pages Answers

dfPagesAnswers.info()

Page keyword tldLang  i = https://www.redhat.com/pt-br/topics/hyperconverged-infrastructure/what-is-hyperconverged-infrastructure o que é computação hiperconvergente 0
get body content encodingutf-8
Links úteis     Nuvem híbrida aberta    Suporte    Desenvolvedores    Parceiros     Comece já a avaliação           Skip to content                 Pesquisar      Enter your keywords           Enter your keywords             Produtos          Soluções          Serviços e suporte          Recursos          Red Hat e o open source           Links úteis          Nuvem híbrida aberta    Suporte    Desenvolvedores    Parceiros     Comece já a avaliação                 Português      Selecione um idioma  简体中文 English Français Deutsch Italiano 日本語 한국어 Português Español     Português     Login  Account        Bem-vindo,  Faça login na sua conta Red Hat        Login  Uma conta da Red Hat dá acesso ao seu perfil, preferências e também aos seguintes serviços, de acordo com o seu status de cliente:  Po

{'answers': ['.',
  'Hiperconvergência é uma abordagem à infraestrutura de TI que consolida os recursos de computação, armazenamento e rede em um sistema unificado.',
  'O que é infraestrutura hiperconvergente? O que é infraestrutura hiperconvergente? Publicado em: 27 de fevereiro de 2020 • %t minutos (tempo de leitura) Copiar URL Visão geral Hiperconvergência é uma abordagem à infraestrutura de TI que consolida os recursos de computação, armazenamento e rede em um sistema unificado.',
  'consolida os recursos de computação, armazenamento e rede em um sistema unificado.',
  'Ela separa as funções de encaminhamento e de controle de rede para criar uma rede que possa ser gerenciada e programada de maneira central. Com a SDN, as equipes de operações de TI controlam o tráfego de rede em topologias complexas por meio de um painel centralizado.',
  'centralizado. Assim, elas não precisam gerenciar cada dispositivo de rede manualmente.',
  'Hiperconvergência é uma abordagem à infraestrutura d

Page keyword tldLang  i = https://www.cisco.com/c/m/pt_br/products/data-center/what-is-hyperconverged-infrastructure.html o que é computação hiperconvergente 1
get body content encodingutf-8
¿Qué es una infraestructura hiperconvergente?     p    ?    m                        O que é infraestrutura hiperconvergente?        A crescente complexidade das operações empresariais modernas requer uma solução que possa simplificar a maneira como implantamos e gerenciamos recursos de data center. A infraestrutura hiperconvergente (HCI, em inglês) elimina a dependência de servidores e redes de armazenamento separados, criando uma plataforma de computação totalmente integrada e interoperável com todo o data center.                Por que a infraestrutura hiperconvergente?  Tornar as operações mais simples, mais rápidas e melhores. Esse é o poder da HCI.         Como funciona?  A HCI combina computação, virtualização, armazenamento e rede em nós unificados que formam um cluster gerenciado centralme

{'answers': ['elimina a dependência de servidores e redes de armazenamento separados, criando uma plataforma de computação totalmente integrada e interoperável com todo o data center.',
  'elimina a dependência de servidores e redes de armazenamento separados,',
  'A infraestrutura hiperconvergente (HCI, em inglês) elimina a dependência de servidores e redes de armazenamento separados, criando uma plataforma de computação totalmente integrada e interoperável com todo o data center.',
  'criando uma plataforma de computação totalmente integrada e interoperável com todo o data center.',
  'plataforma de computação totalmente integrada e interoperável com todo o data center.',
  'elimina a dependência de servidores e redes de armazenamento separados, criando uma plataforma de computação totalmente integrada e interoperável',
  'uma plataforma de computação totalmente integrada e interoperável com todo o data center.',
  'elimina a dependência de servidores e redes de armazenamento separad

Page keyword tldLang  i = https://www.cisco.com/c/pt_pt/solutions/data-center-virtualization/what-is-hyperconverged-infrastructure.html o que é computação hiperconvergente 2
get body content encodingutf-8
Ir para Conteúdo Principal  Ignorar para procurar  Ir para Rodapé                   MENU  FECHAR     Produtos  Suporte  Parceiros  Mais                                       pt    pt                                       Log In      Tem uma conta?      Conteúdo personalizado     Os seus produtos e suporte   Log In   Esqueceu-se da palavra-passe?  Acerca das contas Cisco     Precisa de uma conta?  Criar uma conta   Ajuda                     pt    pt    Seleção de idioma        Produtos    Suporte    Formação e Eventos    Comprar    Parceiros            Solutions Data center     O que é a infraestrutura hiperconvergente (HCI)?                    A infraestrutura hiperconvergente é um sistema concebido com computação, rede, armazenamento e virtualização num único dispositivo com um único

...

Page keyword tldLang  i = https://www.nutanix.com/br/hyperconverged-infrastructure o que é computação hiperconvergente 3
get body content encodingutf-8
Enter your name:                                    Fale conosco  Recursos   Brasil (Português)    Deutschland (Deutsch)  España (Español)  France (Français)  日本 (日本語)  대한민국 (한국어)  中国 (简体中文)  Latinoamérica (Español)  Italia (Italiano)  United Kingdom (English)  United States (English)  Australia (English)  India  台灣 (繁體中文)  Asia Pacific (English)     Log In             O Que Fazemos    Modernize Your Datacenter    Run Any Application at Any Scale    Manage Your Multicloud Environment       Produtos    AOS Storage    AHV Virtualization    Nutanix Kubernetes Engine    Nutanix Disaster Recovery    Flow Network Security    NC2 (Nutanix Cloud Clusters)    Intelligent Operations    Self-Service    Cost Governance    Nutanix Security Central    Files Storage    Objects Storage    Volumes Block Storage    Mine Integrated Backup    VDI    Frame 

{'answers': ['combina hardware comum de data center usando recursos de armazenamento conectados localmente com software inteligente',
  'combina hardware comum de data center usando recursos de armazenamento conectados localmente com software inteligente para criar estruturas flexíveis, que substituem a infraestrutura obsoleta, composta por servidores, redes de armazenamento e matrizes de armazenamento individuais.',
  'A infraestrutura hiperconvergente (HCI) combina hardware comum de data center usando recursos de armazenamento conectados localmente com software inteligente',
  'combina hardware comum de data center usando recursos de armazenamento conectados localmente com software inteligente para criar estruturas flexíveis, que substituem a infraestrutura obsoleta,',
  'A infraestrutura hiperconvergente (HCI) combina hardware comum de data center usando recursos de armazenamento conectados localmente com software inteligente para criar estruturas flexíveis, que substituem a infraes

Page keyword tldLang  i = https://www.binarionet.com.br/infraestrutura-hiperconvergente-veja-o-que-e-e-o-que-ela-pode-fazer-para-o-seu-negocio/ o que é computação hiperconvergente 4
get body content encodingUTF-8
Home  Empresa   Nossos clientes  Prêmios e Certificações  Cases de sucesso  Binário 25 Anos    Soluções e Serviços   XaaS   WaaS  SD-WANaaS  SECaaS    Transformação Digital   Conectividade  Mobilidade  Cloud Computing  Data Center    Inteligência artificial   Automação  Cyber Security  Visibilidade    Infraestrutura   Otimização    Serviços    Parceiros  Blog   E-books    Contato   Contato  Trabalhe conosco  Denúncias  Portal de Garantia do Titular de Dados Pessoais                          Infraestrutura hiperconvergente: veja o que é e o que ela pode fazer para o seu negócio 12 de novembro de 2019 | In Integração | By Grupo Binario          Infraestrutura hiperconvergente — ou HCI, do inglês Hyper-Converged Infrastructure — é um termo que surge inevitavelmente quando as pess

{'answers': ['A infraestrutura hiperconvergente utiliza o software para permitir que um único servidor faça o trabalho de vários servidores tradicionais,',
  'utiliza o software para permitir que um único servidor faça o trabalho de vários servidores tradicionais,',
  'permitir que um único servidor faça o trabalho de vários servidores tradicionais,',
  'software para permitir que um único servidor faça o trabalho de vários servidores tradicionais,',
  'A infraestrutura hiperconvergente utiliza o software para permitir que um único servidor faça o trabalho de vários servidores tradicionais, portanto, ocupa menos espaço em uma sala de servidores.',
  'infraestrutura hiperconvergente utiliza o software para permitir que um único servidor faça o trabalho de vários servidores tradicionais,',
  'utiliza o software para permitir que um único servidor faça o trabalho de vários servidores tradicionais, portanto, ocupa menos espaço em uma sala de servidores.',
  'permitir que um único servidor 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   keyword      100 non-null    object
 1   page         100 non-null    object
 2   position     100 non-null    object
 3   BERT_score   100 non-null    object
 4   source       100 non-null    object
 5   search_date  100 non-null    object
 6   answers      100 non-null    object
 7   starts       100 non-null    object
 8   ends         100 non-null    object
 9   local_probs  100 non-null    object
 10  total_probs  100 non-null    object
dtypes: object(11)
memory usage: 8.7+ KB


In [ ]:
dfPagesAnswers.sort_values(by='BERT_score', ascending=False).head(50)

,keyword,page,position,BERT_score,source,search_date,answers,starts,ends,local_probs,total_probs
99,o que é computação hiperconvergente,https://www.binarionet.com.br/infraestrutura-h...,8,0.986446,Scrap,2022-03-24,utiliza o software para permitir que um único ...,256,264,0.002505,0.965571
89,o que é computação hiperconvergente,https://www.binarionet.com.br/infraestrutura-h...,8,0.986446,Scrap,2022-03-24,software para permitir que um único servidor f...,258,280,0.012868,0.993107
80,o que é computação hiperconvergente,https://www.binarionet.com.br/infraestrutura-h...,8,0.986446,Scrap,2022-03-24,A infraestrutura hiperconvergente utiliza o so...,253,271,0.331338,0.999731
82,o que é computação hiperconvergente,https://www.binarionet.com.br/infraestrutura-h...,8,0.986446,Scrap,2022-03-24,permitir que um único servidor faça o trabalho...,260,271,0.176816,0.999495
83,o que é computação hiperconvergente,https://www.binarionet.com.br/infraestrutura-h...,8,0.986446,Scrap,2022-03-24,software para permitir que um único servidor f...,258,271,0.090323,0.999012
84,o que é computação hiperconvergente,https://www.binarionet.com.br/infraestrutura-h...,8,0.986446,Scrap,2022-03-24,A infraestrutura hiperconvergente utiliza o so...,253,280,0.047203,0.998111
85,o que é computação hiperconvergente,https://www.binarionet.com.br/infraestrutura-h...,8,0.986446,Scrap,2022-03-24,infraestrutura hiperconvergente utiliza o soft...,254,271,0.041978,0.997877
86,o que é computação hiperconvergente,https://www.binarionet.com.br/infraestrutura-h...,8,0.986446,Scrap,2022-03-24,utiliza o software para permitir que um único ...,256,280,0.026664,0.996662
87,o que é computação hiperconvergente,https://www.binarionet.com.br/infraestrutura-h...,8,0.986446,Scrap,2022-03-24,permitir que um único servidor faça o trabalho...,260,280,0.025189,0.996467
88,o que é computação hiperconvergente,https://www.binarionet.com.br/infraestrutura-h...,8,0.986446,Scrap,2022-03-24,um único servidor faça o trabalho de vários se...,262,271,0.014727,0.993972


In [ ]:
dfPagesAnswers.query('position==3').answers[40]

'um sistema concebido com computação, rede, armazenamento e virtualização num único dispositivo com um único ponto de gestão.'

Save Results 

In [ ]:
#  #Save Answers
# dfPagesAnswers.to_csv(os.path.join(base_dir, 'dfPagesAnswersJN.csv'), sep=",", encoding='utf-8', index=False)    



In [ ]:
#Save Bert Scores by page
dfPagesSummary = dfPagesUnique[['keyword', 'page', 'position', 'BERT_score', 'search_date']]
# dfPagesSummary.to_csv(os.path.join(base_dir, 'dfPagesSummaryJN.csv'), sep=",", encoding='utf-8', index=False)         
#  #Save page content in csv and json
# dfPagesUnique.to_csv(os.path.join(base_dir, 'dfPagesUniqueJN.csv'), sep=",", encoding='utf-8', index=False)  #sep ; 
# dfPagesUnique.to_json(os.path.join(base_dir, 'dfPagesUniqueJN.json'))

In [ ]:
dfPagesSummary.sort_values(by='BERT_score', ascending=False).head(50)

,keyword,page,position,BERT_score,search_date
4,o que é computação hiperconvergente,https://www.binarionet.com.br/infraestrutura-h...,8,0.986446,2022-03-24
2,o que é computação hiperconvergente,https://www.cisco.com/c/pt_pt/solutions/data-c...,3,0.968956,2022-03-24
3,o que é computação hiperconvergente,https://www.nutanix.com/br/hyperconverged-infr...,6,0.917998,2022-03-24
1,o que é computação hiperconvergente,https://www.cisco.com/c/m/pt_br/products/data-...,2,0.795364,2022-03-24
0,o que é computação hiperconvergente,https://www.redhat.com/pt-br/topics/hyperconve...,1,0.5826,2022-03-24
